In [4]:
import dlib
import cv2
import numpy as np
import os
from PIL import Image

# The notebook is used for face alignment and for turning images into grayscale.
### This improves performance of our model

In [5]:
model_path="/home/jal/.deepface/weights/vgg_face_weights.h5"
# Load the pre-trained face detection model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../raw_data/shape_predictor_81_face_landmarks.dat')


In [46]:
folder_input_path = "../raw_data/output_imdb_top100/"
folder_output_path = "../raw_data/image_trans/"

file_list = [os.path.join(folder_input_path, file) for file in os.listdir(folder_input_path) if os.path.isfile(os.path.join(folder_input_path, file))]
print(len(file_list))

207


In [ ]:
images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    image = cv2.imread(image_path+'.jpg')
    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the image
    faces = detector(gray)
    # We take only the first detected face, if exist
    if len(faces) == 0:
         # Save the final preprocessed image
        save_path = image_path.replace(folder_input_path,folder_output_path)+".jpg"
        # black and white output:
        cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        print("Image without detected face", image_path)
        continue
    face = faces[0]
    
    # We extract the landmarks   
    landmarks = predictor(image, face)
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

    # Calculate the center of the eyes
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)
    nose = landmarks[30]

    # Calculate the angle between the eyes and the horizontal line
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Rotate the image around the center of the face rectangle
    (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
    center = (x + w // 2, y + h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    if ((aligned_face.shape[0] == 0) or (aligned_face.shape[1] == 0) or (aligned_face.shape[2] == 0)):
        continue
    
    # Re apply the face crop on the aligned image
    gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
    faces_aligned = detector(gray)
    if not faces_aligned:
        continue
    # Modifyers to crop less to capture head fully
    reduce = 0.7
    increase = 1/0.7
    
    x, y, w, h = int(faces_aligned[0].left()*reduce), int(faces_aligned[0].top()*reduce), int(faces_aligned[0].width()*increase), int(faces_aligned[0].height()*increase)
    processed_face = aligned_face[y:y+h, x:x+w]
    if ((processed_face.shape[0] == 0) or (processed_face.shape[1] == 0) or (processed_face.shape[2] == 0)):
        continue
    
    # Save the final preprocessed image
    save_path = image_path.replace(folder_input_path,folder_output_path)+".jpg"
    # black and white output:
    cv2.imwrite(save_path, cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)) #cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(save_path, face_image)
    # print(f"Face image saved: {save_path}")

## To reduce the image size of all images 

In [ ]:
folder_path = "../raw_data/output_imdb_top100/"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    img = cv2.imread(image_path+'.jpg')
    width = int(img.shape[1])
    height = int(img.shape[0])
    target_width = 600
    if width > target_width:
        dim = (target_width, int(height*(target_width/width)))
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
        Image.fromarray(resized).save(image_path+'.jpg')
